# Hyperparameter Search (Optuna)

TPE sampler + MedianPruner over the multimodal RCA model.

In [4]:
import sys
sys.path.insert(0, '/root/lemm')

import os
import json
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
import optuna
from optuna.trial import TrialState

torch.manual_seed(42)
np.random.seed(42)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {DEVICE}')

Device: cuda


In [2]:
# Fixed config (not tuned) - Same structure as train_multimodal_rca.ipynb
BASE_DIR = Path('/root/lemm')
MULTIMODAL_DIR = BASE_DIR / 'core_multimodal_tmp'
METRICS_DIR = BASE_DIR / 'core_metrics_tmp'
LOGS_DIR = BASE_DIR / 'core_logs_tmp'

FIXED_CONFIG = {
    'window_size': 22,
    'n_metrics': 7,
    'bin_seconds': 30,
    'log_embed_dim': 384,
    'epochs': 30,
    'early_stop_patience': 10
}

# Metric file names - Same as train_multimodal_rca.ipynb
METRIC_FILES = [
    'pod_cpu_usage_total.parquet',
    'pod_memory_working_set.parquet',
    'pod_network_rx_bytes.parquet',
    'pod_network_tx_bytes.parquet',
    'pod_latency_server_p95.parquet',
    'pod_latency_client_p95.parquet',
    'pod_workload_ops.parquet',
]

# Split by DAY to guarantee zero temporal overlap (same as train_multimodal_rca.ipynb)
# Nezha scenarios have ~8 min gaps but 10 min windows = 85% overlap within same day
# By splitting by day, we eliminate ALL data leaks between train/val
TRAIN_DAYS = {'20220822', '20230129'}  # Online Boutique day 1 + Train-Ticket day 1
VAL_DAYS = {'20220823', '20230130'}    # Online Boutique day 2 + Train-Ticket day 2

# Excluded scenarios - Same as train_multimodal_rca.ipynb
EXCLUDED = {
    '20220822_nezha_14', '20220822_nezha_22', '20220822_nezha_23',
    '20220823_nezha_21', '20220823_nezha_24',
    '20230130_nezha_15', '20230130_nezha_16',
}

def get_split(scenario_id):
    """Determine if scenario belongs to train or val set."""
    if '_nezha_' in scenario_id:
        date_prefix = scenario_id.split('_nezha_')[0]
        if date_prefix in VAL_DAYS:
            return 'val'
        elif date_prefix in TRAIN_DAYS:
            return 'train'
    # LEMMA scenarios always go to train
    return 'train'

In [3]:
def discover_scenarios():
    """Discover all valid scenarios - same as train_multimodal_rca.ipynb"""
    scenarios = []
    for d in sorted(MULTIMODAL_DIR.iterdir()):
        if not d.is_dir():
            continue
        if d.name in EXCLUDED:
            continue
        if not (d / 'manifest.json').exists():
            continue
        if not (d / 'ground_truth.json').exists():
            continue
        scenarios.append(d.name)
    return scenarios

def load_scenario_data(scenario_id, window_size, n_metrics):
    """Load scenario data - same structure as train_multimodal_rca.ipynb"""
    multimodal_path = MULTIMODAL_DIR / scenario_id
    metrics_path = METRICS_DIR / scenario_id
    logs_path = LOGS_DIR / scenario_id
    
    # Load manifest and ground truth
    with open(multimodal_path / 'manifest.json') as f:
        manifest = json.load(f)
    with open(multimodal_path / 'ground_truth.json') as f:
        gt = json.load(f)
    
    pods = manifest['pods']
    services = manifest['services']
    rc_service = gt['root_cause_service']
    fault_idx = gt['fault_time_idx']
    label = services.index(rc_service) if rc_service in services else 0
    
    # Pod to service mapping (from manifest)
    pod_to_service_idx = manifest.get('pod_to_service_idx', [])
    if not pod_to_service_idx:
        # Fallback: build from pod_to_service dict
        pod_to_svc = manifest.get('pod_to_service', {})
        pod_to_service_idx = []
        for pod in pods:
            svc = pod_to_svc.get(pod, None)
            if svc and svc in services:
                pod_to_service_idx.append(services.index(svc))
            else:
                pod_to_service_idx.append(-1)
    
    # Load metrics - same as train_multimodal_rca.ipynb
    metrics_dict = {}
    for metric_file in METRIC_FILES:
        metric_name = metric_file.replace('pod_', '').replace('.parquet', '')
        path = metrics_path / metric_file
        if path.exists():
            metrics_dict[metric_name] = pd.read_parquet(path)
    
    # Build metrics tensor: (T, n_pods, n_metrics)
    metrics_list = []
    for metric_name in ['cpu_usage_total', 'memory_working_set', 'network_rx_bytes', 
                        'network_tx_bytes', 'latency_server_p95', 'latency_client_p95', 'workload_ops']:
        if metric_name in metrics_dict:
            df = metrics_dict[metric_name]
            metrics_list.append(df.values)  # (T, n_pods)
        else:
            # Fill with NaN if metric missing
            first_metric = list(metrics_dict.values())[0] if metrics_dict else None
            n_timesteps = len(first_metric) if first_metric is not None else window_size
            metrics_list.append(np.full((n_timesteps, len(pods)), np.nan))
    
    metrics = np.stack(metrics_list, axis=-1).astype(np.float32)  # (T, n_pods, n_metrics)
    n_bins = metrics.shape[0]
    
    # Load logs - same as train_multimodal_rca.ipynb
    logs_file = logs_path / 'logs_service_texts.parquet'
    log_texts = {svc: '[N-LGS-DST-TKN-LEZHSA]' for svc in services}
    if logs_file.exists():
        logs_df = pd.read_parquet(logs_file)
        for service in services:
            if service in logs_df.columns:
                service_logs = logs_df[service].fillna('').tolist()
                combined = ' | '.join([l for l in service_logs if l.strip()])
                if combined.strip():
                    log_texts[service] = combined[:512]  # Truncate for MiniLM
    
    return {
        'metrics': metrics, 'n_bins': n_bins, 'fault_idx': fault_idx,
        'pods': pods, 'services': services, 'log_texts': log_texts,
        'label': label, 'rc_service': rc_service,
        'pod_to_service_idx': pod_to_service_idx
    }

In [4]:
class RCADataset(Dataset):
    def __init__(self, scenario_ids, config, mode='train'):
        self.scenario_ids = scenario_ids
        self.config = config
        self.mode = mode
        
        # PRE-LOAD ALL DATA INTO MEMORY (major speedup)
        self.data_cache = {}
        all_services = set()
        for sid in scenario_ids:
            data = load_scenario_data(sid, FIXED_CONFIG['window_size'], FIXED_CONFIG['n_metrics'])
            self.data_cache[sid] = data
            all_services.update(data['services'])
        self.all_services = sorted(all_services)
        print(f"[Dataset] Cached {len(scenario_ids)} scenarios in memory")
    
    def __len__(self):
        return len(self.scenario_ids)
    
    def __getitem__(self, idx):
        sid = self.scenario_ids[idx]
        data = self.data_cache[sid]  # Use cached data instead of loading
        
        metrics = data['metrics'].copy()  # Copy to avoid modifying cache
        window = FIXED_CONFIG['window_size']
        n_bins = data['n_bins']
        fault_idx = data['fault_idx']
        
        if n_bins <= window:
            if n_bins < window:
                pad = window - n_bins
                metrics = np.pad(metrics, ((0, pad), (0, 0), (0, 0)), constant_values=np.nan)
        else:
            jitter = self.config.get('jitter', 0) if self.mode == 'train' else 0
            offset = np.random.randint(-jitter, jitter + 1) if jitter > 0 else 0
            ideal_start = fault_idx - window // 2 + offset
            start = max(0, min(ideal_start, n_bins - window))
            metrics = metrics[start:start + window]
        
        valid_mask = ~np.isnan(metrics).any(axis=-1)
        
        for m in range(metrics.shape[-1]):
            vals = metrics[:, :, m]
            valid = ~np.isnan(vals)
            if valid.sum() > 1:
                mean = np.nanmean(vals)
                std = np.nanstd(vals)
                if std > 1e-8:
                    metrics[:, :, m] = (vals - mean) / std
        
        if self.mode == 'train':
            noise_std = self.config.get('noise_std', 0.0)
            mask_prob = self.config.get('mask_prob', 0.0)
            if noise_std > 0:
                noise = np.random.normal(0, noise_std, metrics.shape).astype(np.float32)
                metrics = metrics + noise * valid_mask[:, :, np.newaxis]
            if mask_prob > 0:
                mask_random = np.random.random(valid_mask.shape) > mask_prob
                valid_mask = valid_mask & mask_random
        
        metrics = np.nan_to_num(metrics, nan=0.0)
        
        return {
            'scenario_id': sid,
            'metrics': torch.tensor(metrics, dtype=torch.float32),
            'metrics_mask': torch.tensor(valid_mask, dtype=torch.bool),
            'log_texts': data['log_texts'],
            'pods': data['pods'],
            'services': data['services'],
            'pod_to_service_idx': data['pod_to_service_idx'],
            'label': data['label'],
            'rc_service': data['rc_service']
        }

In [5]:
class MetricEncoder(nn.Module):
    def __init__(self, n_metrics, d_model, n_heads, n_layers, dropout):
        super().__init__()
        self.input_proj = nn.Linear(n_metrics, d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=n_heads,
            dim_feedforward=d_model * 4, dropout=dropout,
            activation='gelu', batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.output_proj = nn.Linear(d_model, d_model)
    
    def forward(self, x, mask=None):
        B, W, P, M = x.shape
        x = x.permute(0, 2, 1, 3).reshape(B * P, W, M)
        if mask is not None:
            mask = mask.permute(0, 2, 1).reshape(B * P, W)
            src_key_padding_mask = ~mask
        else:
            src_key_padding_mask = None
        x = self.input_proj(x)
        x = self.encoder(x, src_key_padding_mask=src_key_padding_mask)
        x = x.mean(dim=1)
        x = self.output_proj(x)
        x = x.view(B, P, -1)
        return x

class LogEncoder(nn.Module):
    def __init__(self, output_dim, dropout):
        super().__init__()
        self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.sentence_model.eval()
        for p in self.sentence_model.parameters():
            p.requires_grad = False
        self.proj = nn.Sequential(
            nn.Linear(384, output_dim), nn.GELU(),
            nn.Dropout(dropout), nn.Linear(output_dim, output_dim)
        )
    
    def forward(self, log_texts_batch, services_batch):
        all_texts = []
        structure = []
        for log_texts, services in zip(log_texts_batch, services_batch):
            texts = [log_texts.get(s, '[N-LGS-DST-TKN-LEZHSA]') for s in services]
            all_texts.extend(texts)
            structure.append(len(services))
        if not all_texts:
            return []
        with torch.no_grad():
            embeddings = self.sentence_model.encode(all_texts, convert_to_tensor=True)
        # Clone to allow gradients through proj (sentence_model is frozen)
        embeddings = embeddings.clone().detach().requires_grad_(True)
        embeddings = self.proj(embeddings)
        result = []
        idx = 0
        for n in structure:
            result.append(embeddings[idx:idx + n])
            idx += n
        return result

class FusionLayer(nn.Module):
    def __init__(self, metric_dim, log_dim, output_dim, dropout):
        super().__init__()
        self.metric_dim = metric_dim
        self.fusion = nn.Sequential(
            nn.Linear(metric_dim * 2 + log_dim, output_dim), nn.GELU(),
            nn.Dropout(dropout), nn.Linear(output_dim, output_dim)
        )
    
    def forward(self, metric_emb, log_emb, pod_to_service_idx, services_batch):
        B = metric_emb.size(0)
        device = metric_emb.device
        fused = []
        for b in range(B):
            services = services_batch[b]
            n_svc = len(services)
            p2s = pod_to_service_idx[b]
            svc_features = []
            for s_idx in range(n_svc):
                pod_indices = [i for i, si in enumerate(p2s) if si == s_idx]
                if pod_indices:
                    pod_metrics = metric_emb[b, pod_indices]
                    svc_max = pod_metrics.max(dim=0).values
                    svc_mean = pod_metrics.mean(dim=0)
                    svc_metric = torch.cat([svc_max, svc_mean], dim=-1)
                else:
                    svc_metric = torch.zeros(2 * self.metric_dim, device=device)
                svc_log = log_emb[b][s_idx]
                combined = torch.cat([svc_metric, svc_log], dim=-1)
                svc_features.append(combined)
            svc_tensor = torch.stack(svc_features)
            fused.append(self.fusion(svc_tensor))
        return fused

class SimilarityClassifier(nn.Module):
    def __init__(self, embed_dim, temperature):
        super().__init__()
        self.temperature = temperature
        self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.sentence_model.eval()
        for p in self.sentence_model.parameters():
            p.requires_grad = False
        self.proj = nn.Linear(384, embed_dim)
    
    def forward(self, fused_batch, services_batch):
        B = len(fused_batch)
        device = fused_batch[0].device
        max_services = max(len(s) for s in services_batch)
        logits = []
        for b in range(B):
            services = services_batch[b]
            fused = fused_batch[b]
            with torch.no_grad():
                svc_emb = self.sentence_model.encode(services, convert_to_tensor=True)
            # Clone to allow gradients through proj (sentence_model is frozen)
            svc_emb = svc_emb.clone().detach().requires_grad_(True).to(device)
            svc_emb = self.proj(svc_emb)
            fused_norm = F.normalize(fused, dim=-1)
            svc_norm = F.normalize(svc_emb, dim=-1)
            sim = (fused_norm * svc_norm).sum(dim=-1) / self.temperature
            n_svc = len(services)
            if n_svc < max_services:
                pad = torch.full((max_services - n_svc,), -100.0, device=device)
                sim = torch.cat([sim, pad])
            logits.append(sim)
        return torch.stack(logits)

class MultimodalRCA(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.metric_encoder = MetricEncoder(
            n_metrics=FIXED_CONFIG['n_metrics'],
            d_model=config['d_model'], n_heads=config['n_heads'],
            n_layers=config['n_layers'], dropout=config['dropout']
        )
        self.log_encoder = LogEncoder(output_dim=config['d_model'], dropout=config['dropout'])
        self.fusion = FusionLayer(
            metric_dim=config['d_model'], log_dim=config['d_model'],
            output_dim=config['fusion_dim'], dropout=config['dropout']
        )
        self.classifier = SimilarityClassifier(
            embed_dim=config['fusion_dim'], temperature=config['temperature']
        )
    
    def forward(self, metrics, metrics_mask, log_texts, pod_to_service_idx, services):
        metric_emb = self.metric_encoder(metrics, metrics_mask)
        log_emb = self.log_encoder(log_texts, services)
        fused = self.fusion(metric_emb, log_emb, pod_to_service_idx, services)
        logits = self.classifier(fused, services)
        return logits

In [6]:
def collate_fn(batch):
    max_pods = max(s['metrics'].shape[1] for s in batch)
    window = FIXED_CONFIG['window_size']
    n_metrics = FIXED_CONFIG['n_metrics']
    
    metrics_padded, masks_padded, pod_to_service_padded = [], [], []
    for s in batch:
        m = s['metrics']
        mask = s['metrics_mask']
        n_pods = m.shape[1]
        if n_pods < max_pods:
            pad_m = torch.zeros(window, max_pods - n_pods, n_metrics)
            pad_mask = torch.zeros(window, max_pods - n_pods, dtype=torch.bool)
            m = torch.cat([m, pad_m], dim=1)
            mask = torch.cat([mask, pad_mask], dim=1)
        metrics_padded.append(m)
        masks_padded.append(mask)
        p2s = s['pod_to_service_idx'] + [-1] * (max_pods - n_pods)
        pod_to_service_padded.append(p2s)
    
    return {
        'scenario_ids': [s['scenario_id'] for s in batch],
        'metrics': torch.stack(metrics_padded),
        'metrics_mask': torch.stack(masks_padded),
        'log_texts': [s['log_texts'] for s in batch],
        'pods': [s['pods'] for s in batch],
        'services': [s['services'] for s in batch],
        'pod_to_service_idx': pod_to_service_padded,
        'labels': torch.tensor([s['label'] for s in batch], dtype=torch.long),
        'rc_services': [s['rc_service'] for s in batch]
    }

In [7]:
def train_epoch(model, dataloader, optimizer, config):
    model.train()
    total_loss, n_batches = 0, 0
    for batch in dataloader:
        metrics = batch['metrics'].to(DEVICE)
        metrics_mask = batch['metrics_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)
        logits = model(metrics, metrics_mask, batch['log_texts'],
                      batch['pod_to_service_idx'], batch['services'])
        loss = F.cross_entropy(logits, labels, label_smoothing=config.get('label_smoothing', 0.0))
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), config.get('grad_clip_norm', 1.0))
        optimizer.step()
        total_loss += loss.item()
        n_batches += 1
    return total_loss / n_batches

def evaluate(model, dataloader):
    model.eval()
    all_preds, all_labels, all_ranks = [], [], []
    with torch.no_grad():
        for batch in dataloader:
            metrics = batch['metrics'].to(DEVICE)
            metrics_mask = batch['metrics_mask'].to(DEVICE)
            logits = model(metrics, metrics_mask, batch['log_texts'],
                          batch['pod_to_service_idx'], batch['services'])
            preds = logits.argmax(dim=-1)
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(batch['labels'].tolist())
            for i, label in enumerate(batch['labels']):
                sorted_idx = logits[i].cpu().argsort(descending=True)
                rank = (sorted_idx == label).nonzero(as_tuple=True)[0].item() + 1
                all_ranks.append(rank)
    acc1 = sum(p == l for p, l in zip(all_preds, all_labels)) / len(all_labels)
    mrr = sum(1.0/r for r in all_ranks) / len(all_ranks)
    return {'acc1': acc1, 'mrr': mrr}

In [8]:
def objective(trial):
    config = {
        'd_model': trial.suggest_categorical('d_model', [64, 128, 256]),
        'n_heads': trial.suggest_categorical('n_heads', [2, 4, 8]),
        'n_layers': trial.suggest_int('n_layers', 1, 3),
        'fusion_dim': trial.suggest_categorical('fusion_dim', [128, 256]),
        'dropout': trial.suggest_float('dropout', 0.1, 0.5),
        'weight_decay': trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True),
        'label_smoothing': trial.suggest_float('label_smoothing', 0.0, 0.1),
        'grad_clip_norm': trial.suggest_float('grad_clip_norm', 0.5, 2.0),
        'batch_size': trial.suggest_categorical('batch_size', [8, 16, 32]),  # Increased for better GPU utilization
        'learning_rate': trial.suggest_float('lr', 1e-5, 5e-4, log=True),
        'warmup_epochs': trial.suggest_int('warmup_epochs', 3, 15),
        'temperature': trial.suggest_float('temperature', 0.05, 0.2),
        'noise_std': trial.suggest_float('noise_std', 0.0, 0.2),
        'mask_prob': trial.suggest_float('mask_prob', 0.0, 0.2),
        'jitter': trial.suggest_int('jitter', 0, 5)
    }
    
    if config['d_model'] % config['n_heads'] != 0:
        return 0.0
    
    all_scenarios = discover_scenarios()
    train_scenarios = [s for s in all_scenarios if get_split(s) == 'train']
    val_scenarios = [s for s in all_scenarios if get_split(s) == 'val']
    
    train_dataset = RCADataset(train_scenarios, config, mode='train')
    val_dataset = RCADataset(val_scenarios, config, mode='val')
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, collate_fn=collate_fn)
    
    model = MultimodalRCA(config).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    
    best_mrr = 0.0
    patience_counter = 0
    
    for epoch in range(FIXED_CONFIG['epochs']):
        if epoch < config['warmup_epochs']:
            lr = config['learning_rate'] * (epoch + 1) / config['warmup_epochs']
            for pg in optimizer.param_groups:
                pg['lr'] = lr
        
        train_epoch(model, train_loader, optimizer, config)
        results = evaluate(model, val_loader)
        mrr = results['mrr']
        
        trial.report(mrr, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()
        
        if mrr > best_mrr:
            best_mrr = mrr
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= FIXED_CONFIG['early_stop_patience']:
                break
    
    return best_mrr

print('Objective function ready')

Objective function ready


In [9]:
# Optimized: ~8 hours with cache speedup, more random exploration
study = optuna.create_study(
    direction='maximize',
    study_name='multimodal_rca_hpo',
    # More random exploration to cover discrete space well
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=15,       # 15 random trials (up from 10)
        n_warmup_steps=12,         # 12 epochs before pruning each trial
        interval_steps=1           # Check for pruning every epoch
    ),
    sampler=optuna.samplers.TPESampler(
        n_startup_trials=15,       # Match pruner startup
        seed=42                    # Reproducibility
    )
)

print('='*70)
print('OPTUNA HYPERPARAMETER SEARCH')
print('='*70)
print(f'Objective: Maximize MRR')
print(f'Epochs per trial: {FIXED_CONFIG["epochs"]}')
print('='*70)

[I 2025-12-14 21:41:47,809] A new study created in memory with name: multimodal_rca_hpo


OPTUNA HYPERPARAMETER SEARCH
Objective: Maximize MRR
Epochs per trial: 30


In [10]:
# Balanced: 100 trials for ~8 hours total
N_TRIALS = 100
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True, gc_after_trial=True)
print(f'\nCompleted {len(study.trials)} trials')

  0%|          | 0/100 [00:00<?, ?it/s]

[Dataset] Cached 53 scenarios in memory
[Dataset] Cached 45 scenarios in memory


Best trial: 0. Best value: 0.495667:   1%|          | 1/100 [02:51<4:43:12, 171.64s/it]

[I 2025-12-14 21:44:39,265] Trial 0 finished with value: 0.4956665864999198 and parameters: {'d_model': 128, 'n_heads': 2, 'n_layers': 1, 'fusion_dim': 128, 'dropout': 0.3832290311184182, 'weight_decay': 1.1527987128232396e-05, 'label_smoothing': 0.09699098521619944, 'grad_clip_norm': 1.7486639612006325, 'batch_size': 8, 'lr': 3.2877474139911175e-05, 'warmup_epochs': 9, 'temperature': 0.11479175279631737, 'noise_std': 0.058245828039608386, 'mask_prob': 0.1223705789444759, 'jitter': 0}. Best is trial 0 with value: 0.4956665864999198.
[Dataset] Cached 53 scenarios in memory
[Dataset] Cached 45 scenarios in memory


Best trial: 0. Best value: 0.495667:   2%|▏         | 2/100 [04:39<3:39:18, 134.28s/it]

...

[Dataset] Cached 53 scenarios in memory
[Dataset] Cached 45 scenarios in memory


Best trial: 4. Best value: 0.581102:  66%|██████▌   | 66/100 [3:16:03<1:13:52, 130.35s/it]

[W 2025-12-15 00:57:50,921] Trial 66 failed with parameters: {'d_model': 256, 'n_heads': 8, 'n_layers': 2, 'fusion_dim': 256, 'dropout': 0.409307297811371, 'weight_decay': 0.0017725276229324008, 'label_smoothing': 0.01781246010867884, 'grad_clip_norm': 0.7975406800204813, 'batch_size': 32, 'lr': 0.0003032311333761149, 'warmup_epochs': 7, 'temperature': 0.12563714828155803, 'noise_std': 0.14722279099708685, 'mask_prob': 0.08132007347742223, 'jitter': 0} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/root/lemm/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_14370/1721322844.py", line 44, in objective
    train_epoch(model, train_loader, optimizer, config)
  File "/tmp/ipykernel_14370/3145803616.py", line 8, in train_epoch
    logits = model(metrics, metrics_mask, batch['log_texts'],
             ^^^^^^^^^^^^^^^

Best trial: 4. Best value: 0.581102:  66%|██████▌   | 66/100 [3:16:03<1:40:59, 178.23s/it]


KeyboardInterrupt: 

In [11]:
# Celda para analizar importancia
import optuna.importance
import json

print("="*70)
print("ANÁLISIS DE IMPORTANCIA DE PARÁMETROS")
print("="*70)

# Obtener importancias
importances = optuna.importance.get_param_importances(study)

print("\nImportancia de cada parámetro (mayor = más impacto en MRR):\n")
for param, importance in sorted(importances.items(), key=lambda x: x[1], reverse=True):
    bar = "█" * int(importance * 50)
    print(f"{param:20s} {importance:.4f} {bar}")

# Ver parámetros del mejor trial
print("\n" + "="*70)
print("MEJOR TRIAL (#4)")
print("="*70)
best_trial = study.best_trial
print(f"\nMRR: {best_trial.value:.4f}")
print("\nParámetros:")
for k, v in best_trial.params.items():
    print(f"  {k}: {v}")

# Guardar para LODOCV
with open('/root/lemm/best_hyperparams.json', 'w') as f:
    json.dump(best_trial.params, f, indent=2)
print("\n✓ Guardado en /root/lemm/best_hyperparams.json")

# Ver distribución de los mejores trials
print("\n" + "="*70)
print("TOP 10 TRIALS")
print("="*70)
trials_df = study.trials_dataframe()
top10 = trials_df.nsmallest(10, 'value') if 'value' in trials_df else trials_df.head(10)
print(top10[['number', 'value', 'params_d_model', 'params_n_layers', 'params_dropout']].to_string())

ANÁLISIS DE IMPORTANCIA DE PARÁMETROS

Importancia de cada parámetro (mayor = más impacto en MRR):

lr                   0.3165 ███████████████
label_smoothing      0.2782 █████████████
grad_clip_norm       0.1610 ████████
n_heads              0.0544 ██
temperature          0.0512 ██
noise_std            0.0297 █
dropout              0.0293 █
jitter               0.0186 
warmup_epochs        0.0179 
mask_prob            0.0161 
batch_size           0.0108 
fusion_dim           0.0061 
d_model              0.0048 
n_layers             0.0033 
weight_decay         0.0021 

MEJOR TRIAL (#4)

MRR: 0.5811

Parámetros:
  d_model: 64
  n_heads: 4
  n_layers: 3
  fusion_dim: 256
  dropout: 0.27101640734341986
  weight_decay: 1.1919481947918725e-05
  label_smoothing: 0.010789142699330446
  grad_clip_norm: 0.5471437785301014
  batch_size: 8
  lr: 0.00034827974366176894
  warmup_epochs: 6
  temperature: 0.11155743845534447
  noise_std: 0.15111022770860974
  mask_prob: 0.045759633098324495
  jitte

In [ ]:
print('='*70)
print('BEST TRIAL')
print('='*70)

best_trial = study.best_trial
print(f'\nBest MRR: {best_trial.value:.4f}')
print('\nBest hyperparameters:')
for key, value in best_trial.params.items():
    print(f'  {key}: {value}')

pruned = len([t for t in study.trials if t.state == TrialState.PRUNED])
complete = len([t for t in study.trials if t.state == TrialState.COMPLETE])
print(f'\nTrial statistics: Complete={complete}, Pruned={pruned}')

In [ ]:
best_config = {**FIXED_CONFIG, **best_trial.params}
with open('/root/lemm/best_hyperparams.json', 'w') as f:
    json.dump(best_config, f, indent=2)
print('Saved to /root/lemm/best_hyperparams.json')

# Second Round: Focused Search

Based on importance analysis:
- **Explore deeply**: lr, label_smoothing, grad_clip_norm, d_model, n_layers, n_heads, temperature
- **Fix**: dropout, noise_std, jitter, warmup_epochs, mask_prob, batch_size, fusion_dim, weight_decay


In [5]:
# Second round objective with fixed low-importance params
def objective_v2(trial):
    """Focused objective: fix low-importance params, explore important ones."""
    
    # FIXED PARAMS (from trial #4, <3% importance)
    dropout = 0.27
    noise_std = 0.15
    jitter = 0
    warmup_epochs = 6
    mask_prob = 0.05
    batch_size = 8
    fusion_dim = 256
    weight_decay = 1e-5
    
    # EXPLORE: High importance params (fine-grained)
    lr = trial.suggest_float('lr', 1e-4, 5e-4, step=2e-5)  # 21 values
    label_smoothing = trial.suggest_float('label_smoothing', 0.0, 0.05, step=0.005)  # 11 values
    grad_clip_norm = trial.suggest_float('grad_clip_norm', 0.3, 1.0, step=0.1)  # 8 values
    
    # EXPLORE: Architecture (we don't know if low importance is real)
    d_model = trial.suggest_categorical('d_model', [32, 64])
    n_layers = trial.suggest_int('n_layers', 3)
    n_heads = trial.suggest_categorical('n_heads', [4])
    
    # EXPLORE: Medium importance
    temperature = trial.suggest_float('temperature', 0.05, 0.20, step=0.025)  # 7 values
    
    config = {
        **FIXED_CONFIG,
        'd_model': d_model,
        'n_heads': n_heads,
        'n_layers': n_layers,
        'fusion_dim': fusion_dim,
        'dropout': dropout,
        'weight_decay': weight_decay,
        'label_smoothing': label_smoothing,
        'grad_clip_norm': grad_clip_norm,
        'batch_size': batch_size,
        'learning_rate': lr,
        'warmup_epochs': warmup_epochs,
        'temperature': temperature,
        'noise_std': noise_std,
        'mask_prob': mask_prob,
        'jitter': jitter,
    }
    
    # Create datasets
    train_scenarios, val_scenarios = [], []
    for s in discover_scenarios():
        if s not in EXCLUDED:
            if get_split(s) == 'train':
                train_scenarios.append(s)
            else:
                val_scenarios.append(s)
    
    train_ds = RCADataset(train_scenarios, config, mode='train')
    val_ds = RCADataset(val_scenarios, config, mode='val')
    
    train_loader = DataLoader(train_ds, batch_size=config['batch_size'], 
                             shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_ds, batch_size=config['batch_size'],
                           shuffle=False, collate_fn=collate_fn)
    
    # Create model
    model = MultimodalRCA(config).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), 
                                  lr=config['learning_rate'],
                                  weight_decay=config['weight_decay'])
    
    # Training with pruning
    best_mrr = 0.0
    for epoch in range(config['epochs']):
        train_epoch(model, train_loader, optimizer, config)
        metrics = evaluate(model, val_loader)
        mrr = metrics['mrr']
        
        if mrr > best_mrr:
            best_mrr = mrr
        
        # Report for pruning
        trial.report(mrr, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()
    
    return best_mrr

print("Second round objective defined")


Second round objective defined


In [6]:
# Create new study for second round
study_v2 = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(
        seed=42,
        n_startup_trials=10,  # 10 random trials first
    ),
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=10,
        n_warmup_steps=12,
        interval_steps=1
    )
)

# Seed with best trial from round 1
study_v2.enqueue_trial({
    'lr': 0.00035,  # Close to trial #4
    'label_smoothing': 0.01,
    'grad_clip_norm': 0.55,
    'd_model': 64,
    'n_layers': 3,
    'n_heads': 4,
    'temperature': 0.11,
})

print("Study v2 created, seeded with trial #4 values")
print("Starting second round: 50 trials (~2-3 hours)")


[I 2025-12-15 13:38:56,368] A new study created in memory with name: no-name-bdfcaa8e-5eda-4835-8764-0c375b2f19a7


Study v2 created, seeded with trial #4 values
Starting second round: 50 trials (~2-3 hours)


In [7]:
# Run second round - 50 trials focused search
N_TRIALS_V2 = 50
study_v2.optimize(objective_v2, n_trials=N_TRIALS_V2, show_progress_bar=True, gc_after_trial=True)
print(f'\nCompleted {len(study_v2.trials)} trials')


  0%|          | 0/50 [00:00<?, ?it/s]


[W 2025-12-15 13:39:00,118] Trial 0 failed with parameters: {'lr': 0.00035, 'label_smoothing': 0.01, 'grad_clip_norm': 0.55, 'd_model': 64} because of the following error: TypeError("Trial.suggest_int() missing 1 required positional argument: 'high'").
Traceback (most recent call last):
  File "/root/lemm/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_18622/4235438134.py", line 22, in objective_v2
    n_layers = trial.suggest_int('n_layers', 3)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/lemm/.venv/lib/python3.12/site-packages/optuna/_convert_positional_args.py", line 135, in converter_wrapper
    return func(**kwargs)  # type: ignore[call-arg]
           ^^^^^^^^^^^^^^
TypeError: Trial.suggest_int() missing 1 required positional argument: 'high'
[W 2025-12-15 13:39:00,124] Trial 0 failed with value None.


TypeError: Trial.suggest_int() missing 1 required positional argument: 'high'

In [ ]:
# Analyze second round results
print("="*70)
print("SECOND ROUND RESULTS")
print("="*70)

# Importance analysis
importances_v2 = optuna.importance.get_param_importances(study_v2)

print("\nImportancia de parámetros (segunda ronda):\n")
for param, importance in sorted(importances_v2.items(), key=lambda x: x[1], reverse=True):
    bar = "█" * int(importance * 50)
    print(f"{param:20s} {importance:.4f} {bar}")

# Best trial
print("\n" + "="*70)
print("MEJOR TRIAL - SEGUNDA RONDA")
print("="*70)
best_v2 = study_v2.best_trial
print(f"\nMRR: {best_v2.value:.4f}")
print(f"Mejor que ronda 1? {best_v2.value:.4f} vs 0.5811")
print("\nParámetros:")
for k, v in best_v2.params.items():
    print(f"  {k}: {v}")

# Compare with round 1
print("\n" + "="*70)
print("COMPARACIÓN")
print("="*70)
print(f"Ronda 1 mejor: 0.5811 (trial #4)")
print(f"Ronda 2 mejor: {best_v2.value:.4f}")
if best_v2.value > 0.5811:
    print("✓ MEJORAMOS!")
else:
    print("✗ No mejoramos, usar trial #4 de ronda 1")


SECOND ROUND RESULTS

Importancia de parámetros (segunda ronda):

temperature          0.3299 ████████████████
label_smoothing      0.1622 ████████
grad_clip_norm       0.1514 ███████
lr                   0.1449 ███████
d_model              0.1016 █████
n_heads              0.0605 ███
n_layers             0.0496 ██

MEJOR TRIAL - SEGUNDA RONDA

MRR: 0.5788
Mejor que ronda 1? 0.5788 vs 0.5811

Parámetros:
  lr: 0.00036
  label_smoothing: 0.0
  grad_clip_norm: 0.7
  d_model: 64
  n_layers: 3
  n_heads: 2
  temperature: 0.175

COMPARACIÓN
Ronda 1 mejor: 0.5811 (trial #4)
Ronda 2 mejor: 0.5788
✗ No mejoramos, usar trial #4 de ronda 1


In [1]:
# Save best hyperparameters (from whichever round was better)
if best_v2.value > 0.5811:
    # Use round 2 params + fixed params
    final_params = {
        **best_v2.params,
        # Add fixed params back
        'dropout': 0.27,
        'noise_std': 0.15,
        'jitter': 0,
        'warmup_epochs': 6,
        'mask_prob': 0.05,
        'batch_size': 8,
        'fusion_dim': 256,
        'weight_decay': 1e-5,
    }
    source = "round 2"
else:
    # Use round 1 params (trial #4)
    final_params = study.best_trial.params
    source = "round 1 (trial #4)"

# Add fixed config
final_config = {**FIXED_CONFIG, **final_params}

with open('/root/lemm/best_hyperparams.json', 'w') as f:
    json.dump(final_config, f, indent=2)

print(f"✓ Saved best hyperparameters from {source} to /root/lemm/best_hyperparams.json")
print("\nFinal config for LODOCV:")
for k, v in final_config.items():
    print(f"  {k}: {v}")


NameError: name 'best_v2' is not defined